In [1]:
from __future__ import print_function
import numpy as np
import cv2

# local modules
from video import create_capture
from common import clock, draw_str


### CAPTURE VIDEO
cam = cv2.VideoCapture(0)
#set the width and height, and UNSUCCESSFULLY set the exposure time
cam.set(3,1080)
cam.set(4,1024)
cam.set(15, 0.1)


### DEFINE HAAR CASCADE DETECTION FROM COMMAND LINE OR GET DEFAULT
def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),
                                     flags=cv2.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects

def draw_rects(img, rects, color):
    for x1, y1, x2, y2 in rects:
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

import sys, getopt
print(__doc__)


args, video_src = getopt.getopt(sys.argv[2:], '', ['cascade=', 'nested-cascade='])
try:
    video_src = video_src[0]
except:
    video_src = 0
args = dict(args)
fullpath_openCV = '/home/patechoc/anaconda2/share/OpenCV/haarcascades/'
cascade_fn = args.get('--cascade', fullpath_openCV + "haarcascade_frontalface_alt.xml")
nested_fn  = args.get('--nested-cascade', fullpath_openCV + "haarcascade_eye.xml")

### INSTANTIATE THE CLASSIFIERS
cascade = cv2.CascadeClassifier(cascade_fn)
nested = cv2.CascadeClassifier(nested_fn)





while True:
    ret, img = cam.read()
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #
#     gray = cv2.equalizeHist(gray) #
    gray = img
#     cv2.imshow("input",img)
#     key = cv2.waitKey(10)
#     if key == 27:
#         break

    t = clock()
    rects = detect(gray, cascade)  
    vis = img.copy()
    draw_rects(vis, rects, (0, 255, 0))
    if not nested.empty():
        for x1, y1, x2, y2 in rects:
            roi = gray[y1:y2, x1:x2]
            vis_roi = vis[y1:y2, x1:x2]
            subrects = detect(roi.copy(), nested)
            draw_rects(vis_roi, subrects, (255, 0, 0))
    dt = clock() - t

    draw_str(vis, (20, 20), 'time: %.1f ms' % (dt*1000))
    cv2.imshow('facedetect', vis)

    if cv2.waitKey(5) == 27:
        break
cv2.destroyAllWindows()



cv2.destroyAllWindows() 
cv2.VideoCapture(0).release()

Automatically created module for IPython interactive environment
